#### Init

In [1]:
from msal import ConfidentialClientApplication
import pandas as pd
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
import logging, time

def_credential = DefaultAzureCredential()  

## connect to Azure Key Vault securely
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"
kv_client = SecretClient(vault_url=KVUri, credential=def_credential)

## Secret Keys for Bolddesk
account_id       = kv_client.get_secret('netsuite-account-id').value
consumer_key     = kv_client.get_secret('netsuite-consumer-key').value
consumer_secret  = kv_client.get_secret('netsuite-consumer-secret').value
token_id         = kv_client.get_secret('netsuite-token-id').value
token_secret     = kv_client.get_secret('netsuite-token-secret').value


## Secret Keys for Azure App Registration
tenant_id = kv_client.get_secret('azure-tenant-id').value
client_id = kv_client.get_secret('idgov-app-client-id').value
client_secret = kv_client.get_secret('idgov-app-client-secret').value

## google sheet
gcp_dashboard_bot_key = kv_client.get_secret('gcp-dashboard-bot-key').value


In [2]:
import sys
os.chdir('../')

# from module.azure_ad import AzureAD
from module.netsuite import Netsuite 
from module.warehouse import Warehouse
from module.azure_ad import AzureAD
from module.google_sheet import GoogleSheet

## Initialize AD and Bolddesk API Module
ad = AzureAD(tenant_id, client_id, client_secret)
ns = Netsuite(account_id, consumer_key, consumer_secret, token_id, token_secret)
gs = GoogleSheet(secret_key=gcp_dashboard_bot_key)

## initialize warehouse
wh = Warehouse(
    server=os.environ["DB_SERVER"],
    database=os.environ["DB_NAME"],
    credential=def_credential
)

#### Google Sheet

In [4]:
## merge google sheet employees, and Netsuite data
## output back to google sheet
consolidated_employee_sheet_id = '1Z1zd1AmOWhGeDKSfuRmwf9lRFN4-Q89_rlHs54ZomGA'
output_worksheet_name = 'idgov'
input_worksheet_name = 'records'

gs = GoogleSheet(secret_key=gcp_dashboard_bot_key)
wks = gs.pgsc.open_by_key(consolidated_employee_sheet_id).worksheet_by_title(input_worksheet_name)

## prepare left side - google sheet employee records
df1 = wks.get_as_df()\
    .loc[:, ['region','company_name','global_emp_id','emp_name','emp_email','manager_email','dept','job_title','cost_center','emp_type', 'hire_date','emp_status']]\
    .query('emp_status=="Active"')
df1.rename(columns = {
    'region': 'emp_region',
    'company_name': 'emp_company_name',
    'global_emp_id': 'emp_global_id',
    'manager_email': 'emp_manager_email',
    'dept': 'emp_dept',
    'cost_center': 'emp_cost_center',
    'hire_date': 'emp_hire_date',
    'job_title': 'emp_job_title'
}, inplace=True)
df1['emp_hire_date']  = pd.to_datetime(df1.emp_hire_date,  format='%d-%b-%Y',errors='coerce')

## prep right side - netsuite data
df2 = wh.get_table('ns_employees_ad_users').loc[:, ['id','global_empid','email','entityid','giveaccess','title','isinactive','lastmodifieddate','datecreated','regco_name','subsidiary_name','subsidiary_country','supervisor_email','department','ad_userPrincipalName','ad_displayName','ad_employeeId','ad_department','ad_companyName','ad_jobTitle','ad_country','ad_manager_displayName','ad_employeeType','ad_accountEnabled','ad_userType','ad_createdDateTime']]
df2.rename(columns = {
    'id': 'ns_id',
    'global_empid': 'ns_global_id',
    'email': 'ns_email',
    'entityid': 'ns_entityid',
    'giveaccess': 'ns_giveaccess',
    'title': 'ns_title',
    'isinactive': 'ns_isinactive',
    'lastmodifieddate': 'ns_lastmodifieddate',
    'regco_name': 'ns_regco_name',
    'subsidiary_name': 'ns_subsidiary_name',
    'subsidiary_country': 'ns_subsidiary_country',
    'supervisor_email': 'ns_supervisor_email',
    'department': 'ns_department',
    'datecreated': 'ns_datecreated'
}, inplace=True)

## merge left and right
df = pd.merge(df1, df2, left_on='emp_email', right_on="ns_email", how='outer')

## Reformating date columns, so that it stays compatible with Looker
df['ns_datecreated']      = df.ns_datecreated.dt.strftime('%Y-%m-%d')   ## first convert to string
df['ns_lastmodifieddate'] = df.ns_lastmodifieddate.dt.strftime('%Y-%m-%d')
df['ad_createdDateTime']  = df.ad_createdDateTime.dt.strftime('%Y-%m-%d')
df['emp_hire_date']       = df.emp_hire_date.dt.strftime('%Y-%m-%d')
## convert to lower case
df['emp_email'] = df.emp_email.str.lower()
df['ns_email']  = df.ns_email.str.lower()
df['ad_userPrincipalName'] = df.ad_userPrincipalName.str.lower()
# df['emp_name'] = df.emp_name.str.lower()
# df['ns_entityid'] = df.ns_entityid.str.lower()
# df['ad_displayName'] = df.ad_displayName.str.lower()

In [5]:
## merge google sheet employees, and Netsuite data
## output back to google sheet
consolidated_employee_sheet_id = '1Z1zd1AmOWhGeDKSfuRmwf9lRFN4-Q89_rlHs54ZomGA'
output_worksheet_name = 'idgov'
input_worksheet_name = 'records'

gs = GoogleSheet(secret_key=gcp_dashboard_bot_key)
wks = gs.pgsc.open_by_key(consolidated_employee_sheet_id).worksheet_by_title(input_worksheet_name)

## prepare left side - google sheet employee records
df1 = wks.get_as_df()\
    .loc[:, ['region','company_name','notice_period','emp_name','emp_email','manager_email','dept','job_title','cost_center','emp_type', 'hire_date','emp_status']]\
    .query('emp_status=="Active"')

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 913 entries, 0 to 951
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   region         913 non-null    object 
 1   company_name   913 non-null    object 
 2   notice_period  913 non-null    float64
 3   emp_name       913 non-null    object 
 4   emp_email      913 non-null    object 
 5   manager_email  913 non-null    object 
 6   dept           913 non-null    object 
 7   job_title      913 non-null    object 
 8   cost_center    913 non-null    object 
 9   emp_type       913 non-null    object 
 10  hire_date      913 non-null    object 
 11  emp_status     913 non-null    object 
dtypes: float64(1), object(11)
memory usage: 92.7+ KB


In [8]:
df1.hire_date
df1.rename(columns = {
    'region': 'emp_region',
    'company_name': 'emp_company_name',
    'global_emp_id': 'emp_global_id',
    'manager_email': 'emp_manager_email',
    'dept': 'emp_dept',
    'cost_center': 'emp_cost_center',
    'hire_date': 'emp_hire_date',
    'job_title': 'emp_job_title'
}, inplace=True)


In [10]:
df1['emp_hire_date']  = pd.to_datetime(df1.emp_hire_date,  format='%d-%b-%Y',errors='coerce')


In [9]:
consolidated_employee_sheet_id = '1Z1zd1AmOWhGeDKSfuRmwf9lRFN4-Q89_rlHs54ZomGA'
output_worksheet_name = 'idgov'

gs = GoogleSheet(secret_key=gcp_dashboard_bot_key)
wks = gs.pgsc.open_by_key(consolidated_employee_sheet_id).worksheet_by_title('records')

## prepare left side - google sheet employee records
df1 = wks.get_as_df()\
    .loc[:, ['region','company_name','global_emp_id','emp_name','emp_email','manager_email','dept','job_title','cost_center','emp_type', 'hire_date','emp_status']]\
    .query('emp_status=="Active"')

df1.rename(columns = {
    'region': 'emp_region',
    'company_name': 'emp_company_name',
    'global_emp_id': 'emp_global_id',
    'manager_email': 'emp_manager_email',
    'dept': 'emp_dept',
    'cost_center': 'emp_cost_center',
    'hire_date': 'emp_hire_date',
    'job_title': 'emp_job_title'
}, inplace=True)
df1['emp_hire_date']  = pd.to_datetime(df1.emp_hire_date,  format='%d-%b-%Y',errors='coerce')

## prep right side - netsuite data
df2 = wh.get_table('ns_employees_ad_users').loc[:, ['id','global_empid','email','entityid','giveaccess','title','isinactive','lastmodifieddate','datecreated','regco_name','subsidiary_name','subsidiary_country','supervisor_email','department','ad_userPrincipalName','ad_displayName','ad_employeeId','ad_department','ad_companyName','ad_jobTitle','ad_country','ad_manager_displayName','ad_employeeType','ad_accountEnabled','ad_userType','ad_createdDateTime']]
df2.rename(columns = {
    'id': 'ns_id',
    'global_empid': 'ns_global_id',
    'email': 'ns_email',
    'entityid': 'ns_entityid',
    'giveaccess': 'ns_giveaccess',
    'title': 'ns_title',
    'isinactive': 'ns_isinactive',
    'lastmodifieddate': 'ns_lastmodifieddate',
    'regco_name': 'ns_regco_name',
    'subsidiary_name': 'ns_subsidiary_name',
    'subsidiary_country': 'ns_subsidiary_country',
    'supervisor_email': 'ns_supervisor_email',
    'department': 'ns_department',
    'datecreated': 'ns_datecreated'
}, inplace=True)

## merge left and right
df = pd.merge(df1, df2, left_on='emp_email', right_on="ns_email", how='outer')

## Reformating date columns, so that it stays compatible with Looker
df['ns_datecreated']      = df.ns_datecreated.dt.strftime('%Y-%m-%d')   ## first convert to string
df['ns_lastmodifieddate'] = df.ns_lastmodifieddate.dt.strftime('%Y-%m-%d')
df['ad_createdDateTime']  = df.ad_createdDateTime.dt.strftime('%Y-%m-%d')
df['emp_hire_date']       = df.emp_hire_date.dt.strftime('%Y-%m-%d')

## Save To Google Sheet
wks = gs.pgsc.open_by_key(consolidated_employee_sheet_id).worksheet_by_title(output_worksheet_name)
wks.clear(fields='*')
wks.set_dataframe(df, start='A1', escape_formulae=True, extend=True, nan='')

In [11]:
df.ad_userPrincipalName

0       None
1       None
2       None
3       None
4       None
        ... 
1149    None
1150    None
1151    None
1152    None
1153    None
Name: ad_userPrincipalName, Length: 1154, dtype: object

In [7]:
d.name.str.lower()

0                                 nns group elimination
1                                 ntl group elimination
2                             nera networks (s) pte ltd
3                           nera telecommunications ltd
4             nera telecommunications (myanmar) co.,ltd
5                            nera telecommunications as
6                               nera (malaysia) sdn bhd
7                              nera infocom (m) sdn bhd
8                                     pt nera indonesia
9                               nera (thailand) limited
10    nera telecommunications holding (thailand) co....
11                               nera (philippines) inc
12          nera telecommunications (australia) pty ltd
13            nera telecommunications (vietnam) co, ltd
14             nera telecommunications maroc s.a.r.l au
15    nera telecommunications(pakistan) (private) li...
16                       nera telecommunications fz-llc
Name: name, dtype: object

In [62]:
df1 = wks.get_as_df()\
    .loc[:, ['region','company_name','global_emp_id','emp_name','emp_email','manager_email','dept','job_title','cost_center','emp_type', 'hire_date','emp_status']]\
    .query('emp_status=="Active"')

KeyError: "['region', 'company_name', 'global_emp_id', 'manager_email', 'dept', 'job_title', 'cost_center', 'hire_date'] not in index"

#### Script Note

In [ ]:
s = ns.list_scripts()

In [ ]:
e = ns.list_employee_all().loc[:,['id','entityid','email','firstname','lastname','isinactive','title']]
e.columns = [ 'owner_' + x for x in e.columns]
df = pd.merge( s, e, how='left', left_on='owner', right_on='owner_id')

In [ ]:
df = ns.list_login_failure()


In [ ]:
wh.erase ('ns_login_failure')
wh.append('ns_login_failure', df)

In [ ]:
df.info()

In [ ]:
df.script_type.unique()

In [ ]:
q = ''' 
    SELECT
        ScriptNote.internalId,
        ScriptNote.date,
        ScriptNote.scriptType as script_id,
        case
			when script.name IS NOT NULL then 'server'
            when clientScript.name IS NOT NULL then 'client'
            when suitelet.name IS NOT NULL then 'suitelet'
	        when workflowActionScript.name IS NOT NULL then 'workflowActionScript'
            else 'unknown'
        end as script_type,
        clientScript.name || script.name || suitelet.name || workflowActionScript.name as script_name,
        ScriptNote.title,
        ScriptNote.detail
    FROM
        ScriptNote
        LEFT OUTER JOIN script ON ScriptNote.scriptType = script.id		
        LEFT OUTER JOIN clientScript ON ScriptNote.scriptType= clientScript.id
        LEFT OUTER JOIN suitelet ON ScriptNote.scriptType = suitelet.id
        LEFT OUTER JOIN workflowActionScript ON ScriptNote.scriptType = workflowActionScript.id
    WHERE 
	    type = 'ERROR' AND (script.name is not null 
							OR clientScript.name is not null 
							OR suitelet.name is not null 
							OR workflowActionScript.name is not null)	
'''
result = ns.query_all(query=q)
df = pd.DataFrame(result)

In [ ]:
df.info()

#### System Note

In [ ]:
q = ''' 
SELECT TOP 100
	SystemNote.context,
	SystemNote.date,
	SystemNote.field,
	SystemNote.id,
	SystemNote.name,
	SystemNote.newValue,
	SystemNote.oldValue,
	SystemNote.record,
	SystemNote.recordId,
	SystemNote.recordTypeId,
	SystemNote.role,
	SystemNote.type
FROM
	SystemNote
WHERE
	SystemNote.date >= SYSDATE - 90 AND
    SystemNote.field NOT LIKE '%MEDIA%'  AND
	SystemNote.recordTypeId = -268
'''

result = ns.query_all(query=q)
n = pd.DataFrame(result)

In [ ]:
q = ''' 
SELECT
	SystemNote.role as role, 
	SystemNote.recordTypeId as recordTypeId, 
	case 
		when MAX(SystemNote.type) = 1 Then 'View' 
		when MAX(SystemNote.type) = 2 Then 'Create' 
		when MAX(SystemNote.type) = 3 Then 'Edit' 
		else 'Full' 
	end as perm_max
FROM
	SystemNote
WHERE
	SystemNote.date >= SYSDATE - 90 AND
	SystemNote.field NOT LIKE '%MEDIA%' AND
    SystemNote.role IS NOT NULL
GROUP BY
	SystemNote.role, SystemNote.recordTypeId
'''
result = ns.query_all(query=q)
df1 = pd.DataFrame(result)

In [ ]:
ns.list_all_records_definition()

In [ ]:
ns.list_role_record_usage()

In [ ]:
active_role_ids = list(set(ns.list_employee_roles().role_id.to_list() + ns.list_partner_roles().role_id.to_list()))
df = ns.list_role_permissions()

In [ ]:
isin = df.role_id.isin(active_role_ids)

In [ ]:
ns.list_role_permissions()

#### Logins Audit

In [ ]:
df = ns.list_login_audits()

In [ ]:
len(df)

#### Employees

In [4]:
table_name = 'ns_employees_ad_users'
users_df    = wh.get_table('ad_users')\
                .drop(columns=['id','passwordProfile_forceChangePasswordNextSignInWithMfa','passwordProfile_forceChangePasswordNextSignIn'])
users_df.columns = [ 'ad_'+c for c in users_df.columns]
employee_df = ns.list_employees(giveaccess_only=False, refresh=True, active_only=True)
df = pd.merge(employee_df, users_df, how='outer', left_on='email', right_on='ad_userPrincipalName')    

In [ ]:
df

#### License

In [ ]:
df1 = ns.list_employee_license()
df2 = ns.union_employees_partners()

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
employee_roles_df = ns.list_employee_roles()
employee_roles_df['license_type'] = employee_roles_df.role_centertype.apply(lambda x: 'Employee' if x=='EMPLOYEE' else 'Full')
cols = ['id','email','entityid','giveaccess','firstname','middlename','lastname','title','issalesrep','issupportrep','isjobresource','isjobmanager','datecreated','lastmodifieddate','isinactive','regco_name','subsidiary_name','subsidiary_country','supervisor_email','department','license_type']
employee_roles_df = employee_roles_df[cols].drop_duplicates()
# employee_roles_df = r.drop_duplicates()

In [ ]:
r.info()

#### Approval Matrix

In [ ]:
approval_matrix_df = ns.list_approval_matrix()
# employees_df = ns.list_employees().set_index('id')

In [ ]:
approval_matrix_df.info()

In [ ]:
approval_matrix_df['approver_l1_names'] = ''
approval_matrix_df['approver_l2_names'] = ''
approval_matrix_df['approver_l3_names'] = ''
approval_matrix_df['approver_l1_email_names'] = ''
approval_matrix_df['approver_l2_email_names'] = ''
approval_matrix_df['approver_l3_email_names'] = ''

for idx, row in approval_matrix_df.iterrows():

    ## L1 Approvers Renaming
    if row.approver_l1:
        approver_list = [int(x) for x in row.approver_l1.split(',')]
        
        approver_names=[]
        for id in approver_list:
            try: 
                approver_names = approver_names + [employees_df.loc[id, 'entityid']]
            except:
                approver_names = approver_names + [f'Invalid({str(id)})']
        l1_names = ', '.join(approver_names)
        approval_matrix_df.loc[idx, 'approver_l1_names'] = l1_names

    ## L1_Email Approvers Renaming
    if row.approver_l1_email:
        approver_list = [int(x) for x in row.approver_l1_email.split(',')]
        
        approver_names=[]
        for id in approver_list:
            try: 
                approver_names = approver_names + [employees_df.loc[id, 'entityid']]
            except:
                approver_names = approver_names + [f'Invalid({str(id)})']
        l1_names = ', '.join(approver_names)
        approval_matrix_df.loc[idx, 'approver_l1_email_names'] = l1_names

    ## L2 Approvers Renaming
    if row.approver_l2:
        approver_list = [int(x) for x in row.approver_l2.split(',')]
        
        approver_names=[]
        for id in approver_list:
            try: 
                approver_names = approver_names + [employees_df.loc[id, 'entityid']]
            except:
                approver_names = approver_names + [f'Invalid({str(id)})']
        l1_names = ', '.join(approver_names)
        approval_matrix_df.loc[idx, 'approver_l2_names'] = l1_names

    ## L2_Email Approvers Renaming
    if row.approver_l2_email:
        approver_list = [int(x) for x in row.approver_l2_email.split(',')]
        
        approver_names=[]
        for id in approver_list:
            try: 
                approver_names = approver_names + [employees_df.loc[id, 'entityid']]
            except:
                approver_names = approver_names + [f'Invalid({str(id)})']
        l1_names = ', '.join(approver_names)
        approval_matrix_df.loc[idx, 'approver_l2_email_names'] = l1_names        
    ## L3 Approvers Renaming
    if row.approver_l3:
        approver_list = [int(x) for x in row.approver_l3.split(',')]
        
        approver_names=[]
        for id in approver_list:
            try: 
                approver_names = approver_names + [employees_df.loc[id, 'entityid']]
            except:
                approver_names = approver_names + [f'Invalid({str(id)})']
        l1_names = ', '.join(approver_names)
        approval_matrix_df.loc[idx, 'approver_l3_names'] = l1_names

    ## L3_Email Approvers Renaming
    if row.approver_l3_email:
        approver_list = [int(x) for x in row.approver_l3_email.split(',')]
        
        approver_names=[]
        for id in approver_list:
            try: 
                approver_names = approver_names + [employees_df.loc[id, 'entityid']]
            except:
                approver_names = approver_names + [f'Invalid({str(id)})']
        l1_names = ', '.join(approver_names)
        approval_matrix_df.loc[idx, 'approver_l3_email_names'] = l1_names



In [ ]:
cols = ['list_id','list_name','record_id','record_name','description','approver_l1_names','approver_l1_email_names','approver_l2_names','approver_l2_email_names','approver_l3_names','approver_l3_email_names']
approval_matrix_df = approval_matrix_df[cols]

In [ ]:
approval_matrix_df.info()